In [1]:
import enchant
from PIL import Image, ImageEnhance
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
import cv2
import re 
import matplotlib.pyplot as plt
import numpy as np
from imgProcessor.measure.sharpness import parameters
import imutils
from scipy.signal import convolve2d
import math

In [2]:
def accuracyByLevenshteinDistance(textOCR,label):
    #tira espaços, tab e \n
    text = re.sub(r"[\n\t\s]*", "", textOCR)
    labelText = re.sub(r"[\n\t\s]*", "", label)
    
    #calcula a distancia de levenshtein entre os textos
    levDistance = enchant.utils.levenshtein(text, labelText)
    
    #calcula a acurácia fazendo accuracy = n-erros/n
    accuracy = (len(labelText)-levDistance) / len(labelText)
    return accuracy

def blurLevel(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()


def biggest_contour(contours):
        biggest = np.array([])
        max_area = 0
        for i in contours:
            area = cv2.contourArea(i)
            if area > 58000:
                peri = cv2.arcLength(i, True)
                approx = cv2.approxPolyDP(i, 0.015 * peri, True)
                if area > max_area and len(approx) == 4:
                    biggest = approx
                    max_area = area
        return biggest
 

def order_points(pts):
        rect = np.zeros((4, 2), dtype = "float32")

        s = np.sum(pts,axis = 1)

        rect[0] = pts[np.argmin(s)]
        rect[2] = pts[np.argmax(s)]
        
        diff = np.diff(pts, axis = 1)
        rect[3] = pts[np.argmin(diff)]
        rect[1] = pts[np.argmax(diff)]
     
        # return the ordered coordinates
        return rect
       
def getAngle(img):
    image = imutils.resize(img, height = 500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    gray = cv2.bilateralFilter(gray, 20, 30, 30)
    edged = cv2.Canny(gray, 50, 150)
    
  
    cnts = None
    cnts, hierarchy  = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]

    
    screenCnt = None
    screenCnt = biggest_contour(cnts)
    
    if(len(screenCnt)==0 ):
        edged = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,13,0.5)
        cnts = None
        cnts, hierarchy  = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]
        
        screenCnt = None
        screenCnt = biggest_contour(cnts)

    if(len(screenCnt) >0 ):
        cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 1)
        screenCnt = order_points(screenCnt[:,0,:])
        angle = cv2.minAreaRect(screenCnt)[-1]
        angle = 90 - angle if (angle>45) else angle        
        
        return angle
    else:
        return None


def estimate_noise(I):
    
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    Image  = cv2.Sobel(src=I, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) #
    
    H, W = Image.shape

    M = [[1, -2, 1],
        [-2, 4, -2],
        [1, -2, 1]]

    sigma = np.sum(np.sum(np.absolute(convolve2d(Image, M))))

    sigma = sigma * (math.sqrt(math.pi/2) * (1 /  (6 * (W-2) * (H-2))))

    return sigma

In [3]:
accuracyAll = np.array([])
accuracyGood = np.array([])
accuracyMedium = np.array([])
accuracyBad = np.array([])

indexGood = np.array([])
indexMedium = np.array([])
indexBad = np.array([])

for i in range(0,200):
    img = np.array([])
    img = cv2.imread('./images/1'+str(i).zfill(3) +'-receipt.jpg')
    if (i == 25 or i == 34 or i == 41 or i == 45 or i == 63 or i == 109 or i == 137 or i == 169 or i == 194 or i == 192): continue
    
    label = open('./images/1'+str(i).zfill(3)+'-receipt.txt', 'r') 
    labelText = label.read()
    
    imgBlurLevel = blurLevel(img)
    noiseLevel = estimate_noise(img)
    imgAngle = getAngle(img)
    
    text =''
    text = pytesseract.image_to_string(img)
    accuracy = accuracyByLevenshteinDistance(text,labelText)
    accuracyAll = np.append(accuracyAll, accuracy)
    
    #boas - aceitaveis
    if((imgBlurLevel >= 630 and imgBlurLevel <= 1150 and noiseLevel <= 13) or (imgBlurLevel > 1150 and imgBlurLevel <= 1310 and noiseLevel <= 20)):
        if(imgAngle != None):
            if(imgAngle <= 0.59514):
                indexGood = np.append(indexGood,i)
                accuracyGood = np.append(accuracyGood, accuracy)
                
        else:
            indexGood = np.append(indexGood,i)
            accuracyGood = np.append(accuracyGood, accuracy)
    
for i in range(0,200):
    img = np.array([])
    img = cv2.imread('./images/1'+str(i).zfill(3) +'-receipt.jpg')
    if (i == 25 or i == 34 or i == 41 or i == 45 or i == 63 or i == 109 or i == 137 or i == 169 or i == 194 or i == 192): continue
    if(len(indexGood[indexGood == i] ) != 0): continue
    
    label = open('./images/1'+str(i).zfill(3)+'-receipt.txt', 'r') 
    labelText = label.read()
    
    imgBlurLevel = blurLevel(img)
    noiseLevel = estimate_noise(img)
    imgAngle = getAngle(img)
    
    text =''
    text = pytesseract.image_to_string(img)
    accuracy = accuracyByLevenshteinDistance(text,labelText)
    accuracyAll = np.append(accuracyAll, accuracy)
    
    #a serem melhoradas
    if((imgBlurLevel < 350 and noiseLevel < 12) or (imgBlurLevel > 650 and noiseLevel > 25 and noiseLevel < 35)):
        if(imgAngle != None):
            if(imgAngle < 3):
                indexMedium = np.append(indexMedium,i)
                accuracyMedium = np.append(accuracyMedium, accuracy)
        else:
            indexMedium = np.append(indexMedium,i)
            accuracyMedium = np.append(accuracyMedium, accuracy)
    else:
        indexBad = np.append(indexBad,i)
        accuracyBad = np.append(accuracyBad, accuracy)


In [4]:
print(np.mean(accuracyAll))
print(np.mean(accuracyGood))
print(np.mean(accuracyMedium))
print(np.mean(accuracyBad))

0.5918376762112326
0.7923624485144304
0.5860999973063405
0.5738828328573803


In [5]:
import pandas as pd

df1 = pd.read_csv("accuracy_1.csv")
df2 = pd.read_csv("accuracy_2.csv")

accuracyAfter = pd.concat([df1, df2])
# print(accuracyAfter)

accAfter = np.asarray(accuracyAfter['Accuracy'].to_list(), dtype='float')

In [8]:
accMelhorar = np.array([])

for i in range(0,len(indexMedium)):
    name = '1' + str(int(indexMedium[i])).zfill(3)
#     print(accuracyAfter['Image'].to_list())
    if(int(name) not in accuracyAfter['Image'].to_list()): continue
    acc = accuracyAfter['Accuracy'].where(accuracyAfter['Image'] == int(name)).dropna().to_list()[0]
#     print(acc)
    accMelhorar = np.append(accMelhorar, acc)
    
    

In [9]:
print(np.mean(accMelhorar))

0.7412728191930857
